In [3]:
# Importing Pywikibot may produce `Configuration variable ... is defined but unknown` warnings,
# but this is a false positive. It still authenticates properly.
import pywikibot as pwb
import datetime as dt
import re
import json
import time

/home/neilpquinn-wmf/venv/lib/python3.5/site-packages/pywikibot/config2.py:1081: UserWarning: Configuration variable consumer_secret is defined but unknown. Misspelled?
  'Misspelled?'.format(name), UserWarning)
/home/neilpquinn-wmf/venv/lib/python3.5/site-packages/pywikibot/config2.py:1081: UserWarning: Configuration variable access_secret is defined but unknown. Misspelled?
  'Misspelled?'.format(name), UserWarning)
/home/neilpquinn-wmf/venv/lib/python3.5/site-packages/pywikibot/config2.py:1081: UserWarning: Configuration variable access_token is defined but unknown. Misspelled?
  'Misspelled?'.format(name), UserWarning)
/home/neilpquinn-wmf/venv/lib/python3.5/site-packages/pywikibot/config2.py:1081: UserWarning: Configuration variable consumer_token is defined but unknown. Misspelled?
  'Misspelled?'.format(name), UserWarning)


In [4]:
SURVEY_BASE_URL = "https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?"
PREVIEW_BASE_URL = "https://wikimedia.qualtrics.com/jfe/preview/SV_5ABs6WwrDHzAeLr?Q_CHL=preview&"

LIST_BASE = "Community Engagement Insights/MassMessages/Lists/2018/"
TEST_LIST_BASE = "User:Neil P. Quinn/2018 CE Insights/"

In [5]:
frame = pd.read_csv("data/interim/sampling-frame.tsv", sep = "\t")
frame.head()

,user,home_proj,global_edits,proj_group,proj_domain
0,! Bikkit !,dewiki,219.0,dewiki,de.wikipedia.org
1,!NewLondon31,jawiki,46.0,jawiki,ja.wikipedia.org
2,!Silent,ptwiki,20688.8,ptwiki,pt.wikipedia.org
3,"""Colorado Campeão""!",ptwiki,417.0,ptwiki,pt.wikipedia.org
4,"""quasi"" tuttologo",itwiki,117.0,itwiki,it.wikipedia.org


# Set sample sizes

In [7]:
targets = pd.read_table("data/raw/project-group-sample-targets.tsv").set_index("proj_group")
targets.head()

,sample_target
proj_group,
enwiki,900
arwiki,675
zhwiki,450
frwiki,450
eswiki,1125


In [8]:
# Round the highest edit count up to the nearest hundred thousand,
# in order to make a pretty edge for the top bin
top_edge = int(np.ceil(frame["global_edits"].max() / 100000) * 100000)
edit_bins = [10, 30, 150, 600, 1200, 3500, top_edge]
frame["binned_edits"] = pd.cut(frame["global_edits"], edit_bins, right=False)

In [9]:
strata_sizes = frame.groupby(["proj_group", "binned_edits"]).size().unstack()
group_sizes = pd.DataFrame(frame.groupby(["proj_group"]).count()["user"])
strata_group_pcts = np.divide(strata_sizes, group_sizes)
strata_targets = np.ceil(
    # Make sure we don't target fewer than 20 users per bin for privacy reasons
    np.maximum(
        np.multiply(strata_group_pcts, targets),
        20
    )
).apply(np.int64)
strata_samples = np.minimum(strata_targets, strata_sizes)

In [10]:
strata_samples

binned_edits,"[10, 30)","[30, 150)","[150, 600)","[600, 1200)","[1200, 3500)","[3500, 1100000)"
proj_group,,,,,,
arwiki,26,94,95,56,67,72
asia_wps,35,167,183,90,106,96
cee_wps,22,106,121,53,76,74
commons,20,54,85,51,87,165
dewiki,42,289,322,142,183,149
enwiki,28,143,133,49,54,45
eswiki,31,127,131,55,60,48
frwiki,65,299,304,137,165,158
itwiki,23,141,125,55,55,54


In [11]:
strata = pd.DataFrame(
    strata_samples.stack()
).reset_index()

strata.columns = ["proj_group", "edit_bin", "sample_size"]

strata.head()

,proj_group,edit_bin,sample_size
0,arwiki,"[10, 30)",26
1,arwiki,"[30, 150)",94
2,arwiki,"[150, 600)",95
3,arwiki,"[600, 1200)",56
4,arwiki,"[1200, 3500)",67


In [12]:
# Ignore any groups where we couldn't sample the minimum 20 users, because of re-identification risk
# In the 2018 edition, this is only 1 group
idx = strata.index[strata["sample_size"] < 20]

strata.loc[idx, "sample_size"] = 0

# Pull samples

In [13]:
def sample_stratum(stratum_query, sample_size):
    """
    returns the sampled users as a list of (user, proj_domain) tuples
    """ 
    stratum = frame.query(stratum_query)
    stratum_size = len(stratum)
    
    sample = stratum.sample(n = sample_size)[["user", "proj_domain"]]

    sample_list = [
        i for i in sample.itertuples(index = False, name = "user")
    ]

    return sample_list

In [14]:
def sample_row(row):
    query = "proj_group == '{pg}' & binned_edits == '{eb}'".format(
        pg = row.proj_group,
        eb = row.edit_bin
    )
    
    return sample_stratum(query, row.sample_size)

In [15]:
strata["sampled_users"] = strata.apply(sample_row, axis = 1)

In [16]:
strata.head()

,proj_group,edit_bin,sample_size,sampled_users
0,arwiki,"[10, 30)",26,"[(Simo abchir, ar.wikipedia.org), (رائد عزمي ح..."
1,arwiki,"[30, 150)",94,"[(Ostazwalid, ar.wikipedia.org), (Michaelkalex..."
2,arwiki,"[150, 600)",95,"[(أبراهيم غيث, ar.wikipedia.org), (أسامة الساع..."
3,arwiki,"[600, 1200)",56,"[(Assamarraie, ar.wikipedia.org), (عبد الله ال..."
4,arwiki,"[1200, 3500)",67,"[(صفاء, ar.wikipedia.org), (أنور, ar.wikipedia..."


In [17]:
# Do all our strata have the correct number of sampled users?
(strata["sample_size"] == strata["sampled_users"].map(len)).value_counts()

True    108
dtype: int64

## Load corresponding test file

In [18]:
test_strata = pd.read_csv("data/raw/test-strata.tsv", sep = "\t")
test_strata["sampled_users"] = test_strata["sampled_users"].map(json.loads)
test_strata.head()

,proj_group,edit_bin,sampled_users
0,arwiki,"[10, 30)","[[Neil P. Quinn-WMF, ar.wikipedia.org], [EGalv..."
1,asia_wps,"[30, 150)","[[Neil P. Quinn-WMF, ko.wikipedia.org], [EGalv..."
2,cee_wps,"[150, 600)","[[Neil P. Quinn-WMF, uk.wikipedia.org], [EGalv..."
3,commons,"[600, 1200)","[[Neil P. Quinn-WMF, commons.wikimedia.org], [..."
4,dewiki,"[1200, 3500)","[[Neil P. Quinn-WMF, de.wikipedia.org], [EGalv..."


# Link building

In [19]:
edit_codes = {
    "[10, 30)": 1,
    "[30, 150)": 2,
    "[150, 600)": 3,
    "[600, 1200)": 4,
    "[1200, 3500)": 5,
    "[3500, 1100000)": 6
}

proj_codes = {
    "enwiki": "en",
    "arwiki": "ar",
    "zhwiki": "zh",
    "frwiki": "fr",
    "eswiki": "es",
    "ruwiki": "ru",
    "dewiki": "de",
    "ptwiki": "pt",
    "nlwiki": "nl",
    "itwiki": "it",
    "jawiki": "ja",
    "meaf_wps": "me",
    "cee_wps": "ce",
    "asia_wps": "as",
    "weur_wps": "we",
    "commons": "co",
    "wikidata": "dt",
    "other": "ot"
}

In [20]:
def audience(edit_bin):
    if edit_bin <= 3:
        return "AE"
    else:
        return "VAE"

def build_url(row, base_url):
    extra_params = "aud={aud}&prj={prj}&edc={edc}&prjedc={prj}{edc}"
    prj = proj_codes[row.proj_group]
    edc = edit_codes[row.edit_bin]
    
    return base_url + extra_params.format(
        aud = audience(edc),
        prj = prj,
        edc = edc,
    )

def survey_url(row):
    return build_url(row, SURVEY_BASE_URL)

def preview_url(row):
    return build_url(row, PREVIEW_BASE_URL)

In [21]:
strata["survey_url"] = strata.apply(survey_url, axis = 1)
strata["preview_url"] = strata.apply(preview_url, axis = 1)

## Create test links

In [22]:
test_strata["survey_url"] = test_strata.apply(survey_url, axis = 1)
test_strata["preview_url"] = test_strata.apply(preview_url, axis = 1)

# Upload MassMessage lists

## Set up and test Pywikibot access 

In [32]:
# Pywikibot automatically authenticates our user-config.py file
# See https://www.mediawiki.org/wiki/Manual:Pywikibot/user-config.py
pwb_site = pwb.Site("meta", "meta")

def save_page(title, text, comment):
    page = pwb.Page(pwb_site, title)
    page.text = text
    page.save(comment, minor = False)

In [33]:
my_username = pwb_site.username()

test_title = "User:{user}/Pywikibot test".format(user = my_username)
test_text = "Pywikibot saved this page at " + str(dt.datetime.now()) + "."
test_comment = "Test Pywikibot access"

# If the page already exists, this will overwrite it 
save_page(test_title, test_text, test_comment)

Sleeping for 9.6 seconds, 2018-03-29 08:06:11
Page [[User:WMF Surveys/Pywikibot test]] saved


## Set up and test target list generation

In [25]:
def list_text(users):
    text = ""
    
    for user in users:
        text += "* {{{{target | user = {} | site = {}}}}}\n".format(user[0], user[1])
        
    return text

In [26]:
def page_title(row, title_base):
    
    title = title_base + "{pg}{eb}"

    return title.format(
        pg = proj_codes[row.proj_group], 
        eb = edit_codes[row.edit_bin]
    )

def real_page_title(row):
    return page_title(row, LIST_BASE)

def test_page_title(row):
    return page_title(row, TEST_LIST_BASE)

In [27]:
strata["page_title"] = strata.apply(real_page_title, axis = 1)
strata.head()

,proj_group,edit_bin,sample_size,sampled_users,survey_url,preview_url,page_title
0,arwiki,"[10, 30)",26,"[[""Simo abchir"", ""ar.wikipedia.org""], [""\u0631...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...
1,arwiki,"[30, 150)",94,"[[""Ostazwalid"", ""ar.wikipedia.org""], [""Michael...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...
2,arwiki,"[150, 600)",95,"[[""\u0623\u0628\u0631\u0627\u0647\u064a\u0645 ...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...
3,arwiki,"[600, 1200)",56,"[[""Assamarraie"", ""ar.wikipedia.org""], [""\u0639...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...
4,arwiki,"[1200, 3500)",67,"[[""\u0635\u0641\u0627\u0621"", ""ar.wikipedia.or...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...


In [28]:
for row in strata.sample(n = 4).itertuples(index = False):
    print(row.page_title)
    print(list_text(row.sampled_users))

Community Engagement Insights/MassMessages/Lists/2018/ru6


IndexError: string index out of range

## Upload the test list pages

In [29]:
test_strata["page_title"] = test_strata.apply(test_page_title, axis = 1)
test_strata.head()

,proj_group,edit_bin,sampled_users,survey_url,preview_url,page_title
0,arwiki,"[10, 30)","[[""Neil P. Quinn-WMF"", ""ar.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ar1
1,asia_wps,"[30, 150)","[[""Neil P. Quinn-WMF"", ""ko.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/as2
2,cee_wps,"[150, 600)","[[""Neil P. Quinn-WMF"", ""uk.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ce3
3,commons,"[600, 1200)","[[""Neil P. Quinn-WMF"", ""commons.wikimedia.org""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/co4
4,dewiki,"[1200, 3500)","[[""Neil P. Quinn-WMF"", ""de.wikipedia.org""], [""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/de5


In [39]:
for row in test_strata.itertuples(index = False):
    users = row.sampled_users
    
    # Only upload a list if the group has some sampled users
    if len(users) > 0:
        title = row.page_title
        text = list_text(users)
        comment = "Upload a test invitation list for the 2018 [[Community Engagement Insights]] survey"
        
        save_page(title, text, comment)

Page [[User:Neil P. Quinn/2018 CE Insights/ar1]] saved
Sleeping for 9.7 seconds, 2018-03-29 08:11:21
Page [[User:Neil P. Quinn/2018 CE Insights/as2]] saved
Sleeping for 9.7 seconds, 2018-03-29 08:11:31
Page [[User:Neil P. Quinn/2018 CE Insights/ce3]] saved
Sleeping for 9.7 seconds, 2018-03-29 08:11:41
Page [[User:Neil P. Quinn/2018 CE Insights/co4]] saved
Sleeping for 9.7 seconds, 2018-03-29 08:11:51
Page [[User:Neil P. Quinn/2018 CE Insights/de5]] saved
Sleeping for 9.7 seconds, 2018-03-29 08:12:01
Page [[User:Neil P. Quinn/2018 CE Insights/en6]] saved
Sleeping for 9.7 seconds, 2018-03-29 08:12:11
Page [[User:Neil P. Quinn/2018 CE Insights/es1]] saved
Sleeping for 9.7 seconds, 2018-03-29 08:12:21
Page [[User:Neil P. Quinn/2018 CE Insights/fr2]] saved
Sleeping for 9.7 seconds, 2018-03-29 08:12:31
Page [[User:Neil P. Quinn/2018 CE Insights/it3]] saved
Sleeping for 9.7 seconds, 2018-03-29 08:12:41
Page [[User:Neil P. Quinn/2018 CE Insights/ja4]] saved
Sleeping for 9.7 seconds, 2018-03-29

## Upload the list pages

<div class="alert alert-block alert-warning">
    <p>Running the cell below will create a lot of pages on Meta (or overwrite them if they've already been created).</p>
    
    <p>Run it only if you want to do that (but, if you run it accidentally, no major harm done).</p>
</div>

In [40]:
for row in strata.itertuples(index = False):
    users = row.sampled_users
    
    # Only upload a list if the group has some sampled users
    if len(users) > 0:
        title = row.page_title
        text = list_text(users)
        comment = "Upload an invitation list for the 2018 [[Community Engagement Insights]] survey"
        
        # Uncomment this to really create all the pages
        # save_page(title, text, comment)

Page [[Community Engagement Insights/MassMessages/Lists/2018/ar1]] saved
Sleeping for 9.5 seconds, 2018-03-29 09:03:30
Page [[Community Engagement Insights/MassMessages/Lists/2018/ar2]] saved
Sleeping for 9.4 seconds, 2018-03-29 09:03:40
Page [[Community Engagement Insights/MassMessages/Lists/2018/ar3]] saved
Sleeping for 9.4 seconds, 2018-03-29 09:03:50
Page [[Community Engagement Insights/MassMessages/Lists/2018/ar4]] saved
Sleeping for 9.4 seconds, 2018-03-29 09:04:00
Page [[Community Engagement Insights/MassMessages/Lists/2018/ar5]] saved
Sleeping for 9.4 seconds, 2018-03-29 09:04:10
Page [[Community Engagement Insights/MassMessages/Lists/2018/ar6]] saved
Sleeping for 9.4 seconds, 2018-03-29 09:04:20
Page [[Community Engagement Insights/MassMessages/Lists/2018/as1]] saved
Sleeping for 9.5 seconds, 2018-03-29 09:04:30
Page [[Community Engagement Insights/MassMessages/Lists/2018/as2]] saved
Sleeping for 9.2 seconds, 2018-03-29 09:04:41
Page [[Community Engagement Insights/MassMessage

# Save our samples

In [30]:
# Have to serialize our arrays properly so we can reuse them in another file
strata["sampled_users"] = strata["sampled_users"].map(json.dumps)
strata.to_csv("data/interim/strata.tsv", sep = "\t", index = False)

In [31]:
test_strata["sampled_users"] = test_strata["sampled_users"].map(json.dumps)
test_strata.to_csv("data/interim/test-strata.tsv", sep = "\t", index = False)